# Using ChatGPT or Gemini with Python and LangChain

In this notebook you will use ChatGPT or Gemini and LangChain to solve and learn about:

- Langchain chains
- Memory and conversation chains

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

## Install OpenAI and LangChain dependencies

In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Optional: Install LangChain Google Gemini Dependency

Google Gemini API is free (till now). You can get a key [here](https://aistudio.google.com/app/apikey), just need to sign in with your google account. Gemini may not be available fully in EU.

In [ ]:
!pip install langchain-google-genai==0.0.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.2
    Uninstalling google-generativeai-0.3.2:
      Successfully uninstalled google-generativeai-0.3.2


## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['openai_key'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Optional: Load Gemini API credentials

Run this section only if you are using Google Gemini

In [ ]:
import os
import yaml

with open('gemini_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

os.environ["GOOGLE_API_KEY"] = api_creds['gemini_key']

## Load Necessary Dependencies and ChatGPT LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0)

## Optional: Load Google Gemini LLM

Only run the below cell if you don't want to use ChatGPT and want to use Google Gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model="gemini-pro",
                                      convert_system_message_to_human=True)

## Let's look at how to create a basic chain

In [ ]:
PROMPT = "tell me a joke about {topic}"
prompt = ChatPromptTemplate.from_template(PROMPT)
chain = (
         prompt
         |
         model
)

response = chain.invoke({"topic": "bears"})
print(response.content)

Why did the bear break up with his girlfriend? 

Because he couldn't bear the relationship any longer!


In [ ]:
# same code with gemini
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = (
         prompt
         |
         gemini_model
)

response = chain.invoke({"topic": "bears"})
print(response.content)

What do you call a bear with no teeth?

A gummy bear!


### Note: Replace model with gemini_model in any of the code below if you want to use Google Gemini instead of ChatGPT

In [ ]:
# can be used on multiple prompts also
topics = [{'topic': 'AI'}, {'topic': 'Statistics'}]
responses = chain.map().invoke(topics)

In [ ]:
for response in responses:
  print(response.content)
  print('-----')
  print('\n')

Why did the AI cross the road?

To get to the other algorithm.
-----


Why did the statistician get lost in the woods?

Because he took the mean path.
-----




## Basic chains are ad-hoc - No conversation history!

In [ ]:
prompt = ChatPromptTemplate.from_template("{query}")
basic_chain = (
               prompt
               |
              model
)

In [ ]:
response = basic_chain.invoke({"query" : 'What are the first four colors of the rainbow?'})
print(response.content)

The first four colors of the rainbow are red, orange, yellow, and green.


In [ ]:
response = basic_chain.invoke({"query" : 'And the other three?'})
print(response.content) # gives a totally random response

The other three are courage, wisdom, and compassion.


## Let's learn how to add memory to build a conversation chain

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
# create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
# k=3 stores last 3 conversations between you and ChatGPT
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [ ]:
memory.load_memory_variables({}) # shows the conversation history

{'history': []}

In [ ]:
from operator import itemgetter
# creates the conversation chain
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables)
        |
        itemgetter("history")
    )
    |
    prompt
    |
    model
)

In [ ]:
user_input = {'input': 'What are the first four colors of a rainbow'}
response = chain.invoke(user_input)
response.content

'The first four colors of a rainbow are red, orange, yellow, and green.'

In [ ]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
# remember to save your conversation to the memory
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({}) # remembers the conversation history

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.')]}

In [ ]:
user_input = {'input': 'And the last 3?'}
response = chain.invoke(user_input) # uses history of the past conversation to give a better response
response.content

'The last three colors of a rainbow are blue, indigo, and violet.'

In [ ]:
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.'),
  HumanMessage(content='And the last 3?'),
  AIMessage(content='The last three colors of a rainbow are blue, indigo, and violet.')]}